In [124]:
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
import pandas as pd

In [125]:
train_data = pd.read_excel('./data/Ocak2024.xlsx', header=0)
test_data = pd.read_excel('./data/ŞubatİlkHafta.xlsx', header=0)

In [126]:
train_data['id'] = 1


In [127]:
train_data = train_data[['id', "Tarih", "target"]]
test_data = test_data[['id', "Tarih", "target"]][:23]

In [142]:
test_data = pd.read_excel('./data/ŞubatİlkHafta.xlsx', header=0)
test_data['id'] = 1
test_data = test_data[['id', "Tarih", "target"]][:12]

test_data = TimeSeriesDataFrame.from_data_frame(test_data,
                                 id_column="id",
                                 timestamp_column="Tarih"
                                 )

In [128]:
train_data = TimeSeriesDataFrame.from_data_frame(train_data,
                                 id_column="id",
                                 timestamp_column="Tarih"
                                 )


In [135]:
predictor = TimeSeriesPredictor(
    prediction_length=12,
    path="autogluon-m4-hourly",
    target="target",
    eval_metric="MAPE",
    freq="24H",
)


In [136]:
predictor.fit(
    train_data,
    presets="fast_training",
    time_limit=600,
)

Beginning AutoGluon training... Time limit = 600s
AutoGluon will save models to 'autogluon-m4-hourly'
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.1.0: Mon Oct  9 21:28:31 PDT 2023; root:xnu-10002.41.9~6/RELEASE_ARM64_T8112
CPU Count:          8
GPU Count:          0
Memory Avail:       5.63 GB / 16.00 GB (35.2%)
Disk Space Avail:   100.26 GB / 460.43 GB (21.8%)
Setting presets to: fast_training

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAPE,
 'freq': '24H',
 'hyperparameters': 'very_light',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 12,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'target': 'target',
 'time_limit': 600,
 'verbosity': 2}

train_data with frequency 'H' has been re

In [137]:
predictions = predictor.predict(train_data)
predictions.head()

data with frequency 'H' has been resampled to frequency '24H'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


mean          0.1          0.2          0.3  \
item_id timestamp                                                        
1       2024-02-01  1720.614361  1188.323955  1362.130184  1488.126128   
        2024-02-02  1696.048833  1015.363525  1254.783245  1422.361259   
        2024-02-03  1706.554702   869.344565  1169.360318  1370.555476   
        2024-02-04  1692.256673   783.699652  1100.218206  1322.324430   
        2024-02-05  1708.556708   726.495204  1055.716245  1311.148984   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
1       2024-02-01  1611.197876  1720.614361  1832.508852  1945.718683   
        2024-02-02  1565.608467  1696.048833  1835.898378  1980.039373   
        2024-02-03  1550.603351  1706.554702  1863.456763  2035.637782   
        2024-02-04  1518.724325  1692.256673  1877.458620  2068.652676   
        2024-02-05  1518.222235  1708.556708  1903.578498  2123.039314   

                            0.8          0.9  
item_id timestamp                             
1       2024-02-01  2076.320309  2270.908468  
        2024-02-02  2146.524338  2379.207359  
        2024-02-03  2238.541524  2513.983762  
        2024-02-04  2291.983121  2608.631407  
        2024-02-05  2372.618400  2726.227002

In [145]:
import matplotlib.pyplot as plt

predictor.plot(test_data, predictions, quantile_levels=[0.1, 0.9], max_history_length=200, max_num_item_ids=4)


AttributeError: 'TimeSeriesPredictor' object has no attribute 'plot'

In [100]:
predictor.leaderboard(test_data)


data with frequency 'H' has been resampled to frequency '24H'.
Additional data provided, testing on additional data. Resulting leaderboard will be sorted according to test score (`score_test`).


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time_marginal,fit_order
0,RecursiveTabular,-1.021963,-0.300003,0.018579,0.012697,0.246632,5
1,SeasonalNaive,-1.065552,-0.283993,0.016350,0.643993,0.002989,2
2,Naive,-1.065552,-0.283993,0.623994,1.042034,0.003569,1
3,ETS,-1.065552,-0.119601,1.270799,7.448395,0.003100,3
4,WeightedEnsemble,-1.304242,-0.114054,0.024021,0.020128,0.186558,7
5,DirectTabular,-1.304242,-0.114054,0.023694,0.020128,0.236642,6
6,Theta,-1.310247,-0.168488,8.032053,7.076244,0.002478,4
